# Rotator Trajectory Verification

This Jupyter notebook is used to verify the updated rotator trajectory generator by Tekniker.
The test will be performed on summit with the real hardware.
Before that, the Tucson test stand is used to develop the test procedure.
The rotator commandable SAL component (CSC) is used to control the rotator hardware.
The published event and telemetry are stored in the engineering facility database (EFD).

It is noted that the **MTMount** telemetry data is required.
Otherwise, the rotator CSC will put the low-level controller into the **Fault** state.

The **setup** notebook can be used to bring up rotator and mount/CCW.

## Import Modules

In [ ]:
%matplotlib inline
%matplotlib widget
import asyncio
from datetime import datetime
import time
from astropy.time import Time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from lsst.ts import salobj
from lsst.ts import simactuators
from lsst_efd_client import EfdClient

## Declaration of User-defined Functions

It is noted that the test conditions are assigned by Tekniker and it is easier to hard-code the values in functions to simplify the functional call.
The details can see [test plan](https://github.com/lsst-ts/ts_mt_hexRot_simulink/blob/develop/doc/TestPlan.pdf).

In [ ]:
def retrieve_efd_client(is_summit=True):
    """
    Retrieves a client to EFD.

    Parameters
    ----------
    is_summit : bool, optional
        This notebook is running on the summit or not. If not, the returned object will point
        to the test stand at Tucson.

    Returns
    -------
    EfdClient : The interface object between the Nublado and summit/Tucson EFD.
    """
    efd_name = "summit_efd" if is_summit else "tucson_teststand_efd"        
    return EfdClient(efd_name)

In [ ]:
async def print_summary_state(csc):
    """
    Print the summary state of rotator.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    data = await csc.evt_summaryState.aget(timeout=30)
    print(salobj.State(data.summaryState))

In [ ]:
async def print_rotator_position(csc):
    """
    Print the rotator position in degree.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    data = await csc.tel_rotation.aget(timeout=15)
    print(f"Rotator is at {data.actualPosition} degree.")

In [ ]:
async def print_rotator_latest_message(csc):
    """
    Print the latest message of rotator.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    data = await csc.evt_logMessage.aget(timeout=15)
    print(data.message)

In [ ]:
def script_log_start(controller_script, test_title, test_detail):
    """Log the start of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"START -- {test_title} -- LVV-T2539 -- {test_detail} -- Starting Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [ ]:
def script_log_stop(controller_script, test_title, test_detail):
    """Log the stop of test case in script controller.

    Parameters
    ----------
    controller_script : lsst.ts.salobj.controller.Controller
        Script controller.
    test_title : str
        Test title.
    test_detail : str
        Test detail.

    Returns
    -------
    astropy.time.core.Time
        Time object.
    """
    time_now = datetime.now()
    controller_script.log.info(f"STOP -- {test_title} -- LVV-T2539 -- {test_detail} -- Starting Time: {time_now} UTC")
    return Time(time_now.isoformat(), scale="utc", format="isot")

In [ ]:
def generate_tracking_target(position, velocity, period):
    """
    Generate the tracking target.

    Parameters
    ----------
    position : float
        Initial position of target in deg.
    velocity : float
        Velocity of the target in deg/sec.
    period : float
        Period to track in seconds.

    Returns
    -------
    list_position : list
        List of the position.
    list_velocity : list
        List of the velocity.
    """
    # 20 Hz is 0.05 s
    delta_t = 0.05
    number_total_commands = int(period / delta_t)
    
    list_position = [position + velocity * idx for idx in range(0, number_total_commands)]
    list_velocity = [velocity] *  number_total_commands
    return list_position, list_velocity

In [ ]:
async def slew_test(csc, position, est_time_wait, est_time_stop):
    """
    Slew test. The rototator will move in the positive and negative directions.
    It will stop in the final movement.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    est_time_wait : float
        Estimated time to wait of movement in seconds.
    est_time_stop : float
        Estimated time to stop in the final in seconds.
    """
    # Move in one direction
    await csc.cmd_move.set_start(position=position, timeout=60)
    await asyncio.sleep(est_time_wait)
    
    print("arrived 1")
    # Move back to the origin
    await csc.cmd_move.set_start(position=0, timeout=60)
    await asyncio.sleep(est_time_wait)
    print("arrived 2")
    # Move to the other direction
    await csc.cmd_move.set_start(position=-position, timeout=60)
    await asyncio.sleep(est_time_wait)
    print("arrived 3")
    # Move and stop before the end of movement
    await csc.cmd_move.set_start(position=position, timeout=60)
    await asyncio.sleep(est_time_stop)
    await csc.cmd_stop.set_start(timeout=60)

In [ ]:
async def track_target(csc, start_position, end_position, velocity):
    """Track the target.

    Parameters
    ----------
    start_position : float
        Starting position of ramp (deg).
    end_position : float
        Ending position of ramp (deg).
    velocity : float
        Velocity of motion along the ramp (deg/sec).
    """
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator = simactuators.RampGenerator(
        [start_position], [end_position], [velocity], track_advance_time)

    await csc.cmd_trackStart.start(timeout=15)
    for positions, velocities, tai in ramp_generator():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)

In [ ]:
async def track_slew_test(csc):
    """
    Track slew sequence test.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator_1 = simactuators.RampGenerator([0.7], [1.5], [0.068], track_advance_time)
    ramp_generator_2 = simactuators.RampGenerator([0.7], [0], [-0.068], track_advance_time)
    ramp_generator_3 = simactuators.RampGenerator([5], [4.7], [-0.01], track_advance_time)
    ramp_generator_4 = simactuators.RampGenerator([-5], [-4.6], [0.0042], track_advance_time)

    # Track the first target
    await csc.cmd_trackStart.start(timeout=15)

    is_stopped = False
    time_start = time.monotonic()
    for positions, velocities, tai in ramp_generator_1():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)

        # Stop the rotator in the tracking
        delta_time = time.monotonic() - time_start
        if (3 <= delta_time <= 3.5) and (not is_stopped):
            await csc.cmd_stop.set_start(timeout=10)
            
            await asyncio.sleep(1)
            is_stopped = True
            print("check point 1")

        # Allow the tracking again
        if is_stopped:
            await csc.cmd_trackStart.start(timeout=15)
            is_stopped = False
            print("check point 2")
            
    print("check point 3")

    # Track the second target
    for positions, velocities, tai in ramp_generator_2():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)
        
    print("check point 4")

    # Track the third target
    for positions, velocities, tai in ramp_generator_3():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)
        
    print("check point 5")

    # Track the fourth target
    for positions, velocities, tai in ramp_generator_4():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)
        
    print("check point 6")

In [ ]:
async def slew_supersession_test(csc):
    """
    Slew supersession test.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator_1 = simactuators.RampGenerator([60], [62], [0.068], track_advance_time)
    ramp_generator_2 = simactuators.RampGenerator([-60], [-62], [-0.05], track_advance_time)

    # Track the first target
    await csc.cmd_trackStart.start(timeout=15)

    is_stopped = False
    time_start = time.monotonic()
    for positions, velocities, tai in ramp_generator_1():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)

        # Break the tracking of this target to test the supersession of slew
        if (time.monotonic() - time_start) > 8:
            print("check point 1")
            break
            
    print("check point 2")

    # Track the second target
    for positions, velocities, tai in ramp_generator_2():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)
        
    print("check point 3")

In [ ]:
async def track_slew_multiple_stop_test(csc):
    """
    Track slew sequence test with multiple stops.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the rotator CSC.
    """
    # 20 Hz is 0.05 ms
    track_advance_time = 0.05
    ramp_generator_1 = simactuators.RampGenerator([0.7], [1.5], [0.068], track_advance_time)
    ramp_generator_2 = simactuators.RampGenerator([0.7], [0], [-0.068], track_advance_time)

    # Track the first target
    await csc.cmd_trackStart.start(timeout=15)

    is_stopped = False
    time_start = time.monotonic()
    for positions, velocities, tai in ramp_generator_1():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)

        # Stop the rotator in the tracking
        delta_time = time.monotonic() - time_start
        if ((3 <= delta_time <= 3.5) or (6 <= delta_time <= 6.5)) and (not is_stopped):
            await csc.cmd_stop.set_start(timeout=10)
            
            await asyncio.sleep(5)
            is_stopped = True
            print("check point 1")

        # Allow the tracking again
        if is_stopped:
            await csc.cmd_trackStart.start(timeout=15)
            is_stopped = False
            print("check point 2")
            
    print("check point 3")

    # Track the second target
    for positions, velocities, tai in ramp_generator_2():
        await csc.cmd_track.set_start(
            angle=positions[0],
            velocity=velocities[0],
            tai=tai,
            timeout=5,
        )
        await asyncio.sleep(0.1)
        
    print("check point 4")

In [ ]:
async def plot_path_generator(efd_client, time_start, time_end, filename=None):
    """
    Plot the data of path generator.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.
    filename : str or None
        Save the data to csv file if filename is not None. (the default is None.)
    """
    # Get the data from EFD
    data_path = await efd_client.select_time_series(
        "lsst.sal.MTRotator.rotation", 
        fields=["demandPosition", "demandVelocity", "demandAcceleration", "actualPosition", "actualVelocity", "timestamp"], 
        start=time_start, 
        end=time_end)

    data_torque = await efd_client.select_time_series(
        "lsst.sal.MTRotator.motors", 
        fields=["torque0", "torque1"], 
        start=time_start, 
        end=time_end)
    
    # Realign the time origin to 0
    time_operation = np.array(data_path.timestamp.subtract(data_path.timestamp[0]))

    # Calculate the acceleration and jerk
    # Note the Simulink model had applied the low-pass filter for the calculation of velocity already 
    acceleration, jerk = calc_acceleration_and_jerk(time_operation, data_path.actualVelocity)

    # Plots
    plt.figure()
    plt.subplot(7, 1, 1)
    plt.plot(time_operation, data_path.demandPosition)
    plt.plot(time_operation, data_path.actualPosition, "x-")
    plt.title("Path Generator")
    plt.ylabel("P")

    plt.subplot(7, 1, 2)
    plt.plot(time_operation, data_path.demandPosition - data_path.actualPosition)
    plt.ylabel("dP")

    plt.subplot(7, 1, 3)
    plt.plot(time_operation, data_path.demandVelocity)
    plt.plot(time_operation, data_path.actualVelocity, "x-")
    plt.ylabel("V")

    plt.subplot(7, 1, 4)
    plt.plot(time_operation, data_path.demandVelocity - data_path.actualVelocity)
    plt.ylabel("dV")

    plt.subplot(7, 1, 5)
    plt.plot(time_operation, data_path.demandAcceleration)
    plt.plot(time_operation, acceleration, "x-")
    plt.ylabel("A")

    plt.subplot(7, 1, 6)
    plt.plot(time_operation, data_path.demandAcceleration - acceleration)
    plt.ylabel("dA")

    plt.subplot(7, 1, 7)
    plt.plot(time_operation, jerk)
    plt.ylabel("J")
    plt.xlabel("Time (s)")

    plt.figure()
    plt.subplot(2, 1, 1)
    plt.plot(time_operation, data_torque.torque0, "r")
    plt.plot(time_operation, data_torque.torque1, "b-")
    plt.title("Motor Torque")
    plt.ylabel("T")
    
    plt.subplot(2, 1, 2)
    plt.plot(time_operation, data_torque.torque0 - data_torque.torque1)
    plt.xlabel("Time (s)")
    plt.ylabel("dT")
    
    print("Units: P (deg), V (deg/s), A (deg/s^2), J (deg/s^3), Torque T (N * m)")
    
    # Save the file
    if filename is not None:
        data_path.to_csv(path_or_buf=f"path_{filename}.csv")
        data_torque.to_csv(path_or_buf=f"torque_{filename}.csv")

In [ ]:
def calc_acceleration_and_jerk(time, velocity):
    """
    Calculate the acceleratioin and jerk.

    Parameters
    ----------
    time : numpy.ndarray
        Time in seconds.
    velocity : pandas.core.series.Series
        velocity (deg/sec).

    Returns
    ----------
    acceleration : numpy.ndarray
        Acceleration in deg/sec^2.
    jerk : numpy.ndarray
        Jerk in deg/sec^3.
    """

    acceleration = np.gradient(velocity, time)
    jerk = np.gradient(acceleration, time)

    return acceleration, jerk

In [ ]:
async def get_event_data(efd_client, time_start, time_end):
    """
    Get the event data.

    Parameters
    ----------
    efd_client : lsst_efd_client.efd_helper.EfdClient
        EFD client.
    time_start : astropy.time.core.Time
        Start time.
    time_end : astropy.time.core.Time
        End time.

    Returns
    -------
    data_controller_state : pandas.core.frame.DataFrame
        Data of the controller state event.
    data_in_position : pandas.core.frame.DataFrame
        Data of the inPosition event.
    data_tracking : pandas.core.frame.DataFrame
        Data of the tracking event.
    """

    # Get the data from EFD
    data_controller_state = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_controllerState", 
        fields=["controllerState", "enabledSubstate"], 
        start=time_start,
        end=time_end)
    
    # The inPosition field is a combination of Flags_moveSuccess and Flags_trackingSuccess in Simulink telemetry
    data_in_position = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_inPosition", 
        fields="inPosition",
        start=time_start,
        end=time_end)
    
    # The tracking field is Flags_trackingSuccess in Simulink telemetry
    # The lost field is Flags_trackingLost in Simulink telemetry
    # The noNewCommand field is Flags_noNewTrackCmdError in Simulink telemetry
    data_tracking = await efd_client.select_time_series(
        "lsst.sal.MTRotator.logevent_tracking", 
        fields=["tracking", "lost", "noNewCommand"],
        start=time_start,
        end=time_end)

    # Add the column of time difference in seconds
    add_column_time_differece(data_controller_state, time_start)
    add_column_time_differece(data_in_position, time_start)
    add_column_time_differece(data_tracking, time_start)

    return data_controller_state, data_in_position, data_tracking

In [ ]:
def add_column_time_differece(data_frame, time_start):
    """
    Add the column of time difference in seconds.

    Parameters
    ----------
    data_frame : pandas.core.frame.DataFrame
        Data frame.
    time_start : astropy.time.core.Time
        Start time.
    """
    if (len(data_frame.values) != 0):
        timestamp_start = pd.Timestamp(time_start.value, tz="utc")
        diff_time = data_frame.index - pd.Timestamp(time_start.value, tz="utc")
        diff_time_sec = [element.seconds + element.microseconds * 1e-6 for element in diff_time]
        data_frame["time_difference"] = diff_time_sec

## Instantiate the EFD and CSC

Need to make sure **ospl** is running for the data distribution system (DDS) communication.

In [ ]:
is_summit = True
efd_client = retrieve_efd_client(is_summit=is_summit)
domain = salobj.Domain()
rotator = salobj.Remote(domain, "MTRotator")
await rotator.start_task
await rotator.cmd_setLogLevel.set_start(level=10)

**MTMount** controller is used to run the camera cable wrapper (CCW).

In [ ]:
mtmount = salobj.Remote(domain, "MTMount")
await mtmount.start_task

state_mtmount = await mtmount.evt_summaryState.aget(timeout=10)
if state_mtmount != salobj.State.ENABLED:
    await salobj.set_summary_state(mtmount, salobj.State.ENABLED, timeout=10)

await mtmount.cmd_enableCameraCableWrapFollowing.set_start()

Put the rotator CSC into **Enabled** state before the test.
At this moment, we need to do the workaround before DM-33967.
At Tucson teststand, we can use *tts_test.yaml* to workaround the following error of rotator and camera cable wrapper (CCW).

In [ ]:
# "tts_test.yaml" is the configuration file at Tucson teststand to increase the tolerance in the
# difference between the rotator and CCW
# settingsToApply = "" if is_summit else "tts_test.yaml" 
# await salobj.set_summary_state(
#     rotator, salobj.State.ENABLED,
#     timeout=30,
#     settingsToApply=settingsToApply
# )

The followings are the code to workaround the rotator CSC telemetry problem.
This should not be needed after people deployed the correct version of rotator CSC on summit.

In [ ]:
await rotator.cmd_start.set_start(timeout=15)


In [ ]:
await rotator.cmd_standby.set_start(timeout=30)


In [ ]:
await rotator.cmd_start.set_start(timeout=15)
await asyncio.sleep(10)

In [ ]:
await rotator.cmd_enable.set_start(timeout=30)

In [ ]:
await rotator.cmd_disable.set_start(timeout=30)

### Check the rotator condition

After each test case, execute this cell to get the **time_now**, which is used in the query of EFD data.

In [ ]:
await print_rotator_position(rotator)
await print_summary_state(rotator)
await print_rotator_latest_message(rotator)

### Put the rotator back to origin

Before each test, put the rotator back to origin.
This is easier for the data analysis.

In [ ]:
await rotator.cmd_move.set_start(position=0, timeout=60)

### Setup the Script

In [ ]:
time_start_verification = datetime.now()
test_title = "Rotator New Trajectory Verification Test"
script = salobj.Controller("Script", index=10)

# Wait 10 seconds may help with DDS problems; closing all other kernels may help too
await asyncio.sleep(10)
print(f"Time to start the verification is {datetime.now() - time_start_verification} [s]")

#### Test Case (small slewing test)

Test the positive and negative directions.

In [ ]:
time_start = script_log_start(script, test_title, "small slewing test")

In [ ]:
await slew_test(rotator, 0.7,5, 0.1)

In [ ]:
time_stop = script_log_stop(script, test_title, "small slewing test")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="small_slew_test")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (long slew test + constant speed test)

Test the positive and negative directions.
The behavior of trajectory generator should be the same for these two tests.
Note the CSC does not support the command of constant speed move.

In [ ]:
time_start = script_log_start(script, test_title, "long slew / constant speed test")

In [ ]:
await slew_test(rotator, 90, 35, 10)

In [ ]:
time_stop = script_log_stop(script, test_title, "long slew / constant speed test")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="long_slew_constant_speed")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (rotation range test)

Note the CSC does not support the command of constant speed move.

In [ ]:
time_start = script_log_start(script, test_title, "rotation range test")

The configuration file of controller can update the following parameters: Parameters_upperPosLimit_deg, Parameters_lowerPosLimit_deg, Parameters_PositionErrorThresh, Parameters_FollowingErrorThreshold_deg, Parameters_trackingSuccessPositionThreshold_deg, and Parameters_trackingLostTimeout_sec in the Simulink model.

In [ ]:
# Test the move position is out of range
# Need to modify the configuration file in the rotator controller to have the max/min of +/-45 degree
# before the test
try:
    await rotator.cmd_move.set_start(position=46, timeout=60)
except salobj.AckError as error:
    print(error)

try:
    await rotator.cmd_move.set_start(position=-46, timeout=60)
except salobj.AckError as error:
    print(error)

In [ ]:
time_stop = script_log_stop(script, test_title, "rotation range test")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="rotation_range")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (position accuracy)

The **slewingAndTracking** operation will be tested.
Note the rotator will transition to Fault state in the final if there is no new track command.
This behaviour can be used to verify the test case of "no tracking commands".
In addition, the trajectory generator will do the extrapolation in a small time period before the transition to **Fault** state.

Target is moving in the positive direction.

In [ ]:
time_start = script_log_start(script, test_title, "position accuracy (positive)")

In [ ]:
await track_target(rotator, 30, 30.5, 0.03)

In [ ]:
time_stop = script_log_stop(script, test_title, "position accuracy (positive)")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="position_accuracy_positive")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

Target is moving in the negative direction.

In [ ]:
time_start = script_log_start(script, test_title, "position accuracy (negative)")

In [ ]:
await track_target(rotator, -30, -31.1, -0.068)

In [ ]:
time_stop = script_log_stop(script, test_title, "position accuracy (negative)")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="position_accuracy_negative")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (tracking / slewing sequence test)

Track two targets and stop in between.

In [ ]:
time_start = script_log_start(script, test_title, "tracking/slewing sequence test")

In [ ]:
await track_slew_test(rotator)

In [ ]:
time_stop = script_log_stop(script, test_title, "tracking/slewing sequence test")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="tracking_slewing_sequence")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (tracking without initial slewing)

No initial slewing is applied to the rotator.

In [ ]:
time_start = script_log_start(script, test_title, "tracking w/o initial slewing")

In [ ]:
data = await rotator.tel_rotation.aget(timeout=15)
await track_target(rotator, data.actualPosition, data.actualPosition + 0.5, 0.068)

In [ ]:
time_stop = script_log_stop(script, test_title, "tracking w/o initial slewing")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="tracking_no_initial_slewing")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (slewing supersession)

Test the slewing supersession capability of the rotator.

In [ ]:
time_start = script_log_start(script, test_title, "slewing supersession")

In [ ]:
await slew_supersession_test(rotator)

In [ ]:
time_stop = script_log_stop(script, test_title, "slewing supersession")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="slewing_supersession")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking

#### Test Case (tracking / slewing with multiple stops)

In [ ]:
time_start = script_log_start(script, test_title, "tracking/slewing with multiple stops")

In [ ]:
await track_slew_multiple_stop_test(rotator)

In [ ]:
time_stop = script_log_stop(script, test_title, "tracking/slewing with multiple stops")

In [ ]:
await plot_path_generator(efd_client, time_start, time_stop, filename="tracking_slewing_multiple_stops")

In [ ]:
data_controller_state, data_in_position, data_tracking = await get_event_data(
    efd_client, time_start, time_stop)

In [ ]:
data_controller_state

In [ ]:
data_in_position

In [ ]:
data_tracking